## Variables

### Import des librairies

In [2]:
import os
from rdflib import Graph
import code.filemanagement as fm
import code.wikidata as wd
import code.ontorefine as otr
import code.graphdb as gd

### Définition des variables globales

In [14]:
# Fichiers existants
addr_ont_file_name = "address_ont.ttl"
ev_ont_file_name = "events_ont.ttl"

# Fichiers créés durant le processus
export_file_name = "addresses-temp.ttl"
out_file_name = "addresses.ttl"
local_config_file_name = "config_repo.ttl"

tmp_folder_name = "tmp_files"
source_folder_name = "sources"
mapping_folder_name = "mappings"
wikidata_folder_name = "wikidata"

project_name = "voies_paris_hist" 
graphdb_url = "http://localhost:7200"

wd_graph_name = "wikidata"
addr_graph_name = "local_street"

### Traitement des variables globales

* Obtention des chemins absolus des fichiers à partir des chemins relatifs donnés dans la section précédente
* Création d'un dossier temporaire s'il n'existe pas encore pour stocker des fichiers à vocation d'être supprimés

In [15]:
tmp_folder = os.path.abspath(tmp_folder_name)
wikidata_folder = os.path.join(tmp_folder, wikidata_folder_name)
source_folder = os.path.abspath(source_folder_name)
mapping_folder = os.path.abspath(mapping_folder_name)

fm.create_folder_if_not_exists(tmp_folder)
fm.create_folder_if_not_exists(wikidata_folder)

local_config_file = os.path.join(tmp_folder, local_config_file_name)
addr_ont_file = os.path.abspath(addr_ont_file_name)
ev_ont_file = os.path.abspath(ev_ont_file_name)

## Gestion du répertoire local

### Création du répertoire local dans GraphDB
Pour que la création marche, il faut que GraphDB soit lancé et donc que l'URI donné par `graphdb_url` fonctionne. Si le répertoire existe déjà, rien n'est fait

In [16]:
gd.create_config_local_repository_file(local_config_file, project_name)
gd.create_repository_from_config_file(graphdb_url, local_config_file)

{"message":"Repository voies_paris_hist already exists."}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1649    0    57  100  1592   1773  49540 --:--:-- --:--:-- --:--:-- 68708


### Vidage du répertoire local
Le répertoire dont l'id est `project_name` pour y stocker les données récupérées, cela est utile si le répertoire existait déjà.

In [17]:
gd.clear_repository(graphdb_url, project_name)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


### Import des ontologies

In [18]:
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, addr_ont_file)
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, ev_ont_file)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19366    0     0  100 19366      0  70695 --:--:-- --:--:-- --:--:-- 72531
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12901    0     0  100 12901      0  52494 --:--:-- --:--:-- --:--:-- 53754


## Import des données de Wikidata
Récupération des fichiers ttl d'éléments de Wikidata (voies, quartiers, arrondissements de Paris). Ensuite ces fichiers sont importés dans le répertoire local.

### Récupération des éléments de Wikidata
Pour chaque élément Wikidata décrivant une rue de Paris, on extrait un fichier `ttl` le décrivant.
Ce processus peut prendre du temps (environ 1 heure et demie) du fait du nombre d'éléments à récupérer (presque 7000).

#### Création des requêtes
Création des requêtes pour récupérer les données liées aux voies de Paris et une autre pour avoir les zones de la ville (quartiers administratifs et arrondissements)

In [20]:
qid_variable = "qid"

query_streets = f"""
SELECT DISTINCT ?{qid_variable}
WHERE {{
{{?{qid_variable} p:P361 [ps:P361 wd:Q16024163].}}
UNION
{{?{qid_variable} p:P361 [ps:P361 wd:Q107311481].}}
}}
ORDER BY ?{qid_variable}
"""

query_areas = f"""
SELECT DISTINCT ?qid WHERE {{
  {{BIND (wd:Q90 AS ?{qid_variable}) }}
  UNION{{?{qid_variable} p:P31 [ps:P31 wd:Q252916].}}
  UNION{{?{qid_variable} p:P31 [ps:P31 wd:Q702842]; p:P131 [ps:P131 wd:Q90].}}
}}
"""

#### Lancement des requêtes

In [21]:
# # Décommenter ces lignes pour récupérer les données

# # Récupération des voies de Paris
# wd.get_ttl_files_from_wikidata_query(query_streets, qid_variable, wikidata_folder, flavor="dump")

# #Récupération des zones de Paris
wd.get_ttl_files_from_wikidata_query(query_areas, qid_variable, wikidata_folder, flavor="dump")

### Import des fichiers `ttl` venant de Wikidata
Importer tous les fichiers ttl dans un graphe nommé défini par `wd_graph_name` dans le répertoire (une vingtaine de minutes pour 7000 fichiers)

In [22]:
gd.upload_ttl_folder_in_graphdb_repository(wikidata_folder, graphdb_url, project_name, wd_graph_name)

100 12759    0     0  100 12759      0  71335 --:--:-- --:--:-- --:--:-- 73751
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13237    0     0  100 13237      0  24245 --:--:-- --:--:-- --:--:-- 24467
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12078    0     0  100 12078      0  65003 --:--:-- --:--:-- --:--:-- 67100
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8345    0     0  100  8345      0  44177 --:--:-- --:--:-- --:--:-- 46620
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11897    0     0  100 11897      0  60084 --:--:

## Traitements pour créer / importer des données

### Requêtes SPARQL pour construire le graphe des rues de Paris à partir de Wikidata

:warning: Impossible de chaîner les requêtes sur le endpoint de Wikidata, d'où la séparation de celles-ci sous plusieurs variables

Note : `wd:Q16024163` décrit l'élément `réseau viaire de Paris` donc le triplet `?street p:P361 [ps:P361 wd:Q16024163].` permet de sélectionner les élements qui font partie de ce réseau. `wd:Q107311481` est similaire à la différence qu'il décrit l'élément `anciennes voies de Paris`.

* `query_1` : sélection des voies de Paris avec leur label, leurs potentiels alias, ainsi que leur localisation (arrondissements et/ou quartiers)
* `query_2` : sélection des voies de Paris avec l'ensemble de leur nom officiel avec si elles existent les dates de début et de fin de validité du nom officiel pour la voie
* `query_3` : sélection (si elles existent) des dates de création et de disparition des voies de Paris
* `query_4` : sélection des arrondissements de Paris avec leur label, leurs potentiels alias et leur localisation (qui est normalement Paris, ie `wd:Q90`)
* `query_5` : similaire à `query_4` mais repose sur les quartiers de Paris (leur localisation est normalement un arrondissement)
* `query_6` : récupération de données sur la ville de Paris (label, alias)


In [5]:
# Récupération de l'URI des graphes à partir de leur nom
wd_graph_name_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, wd_graph_name)
addr_graph_name_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, addr_graph_name)

# Récupération des préfixes pour les requêtes
prefixes = gd.get_repository_prefixes(graphdb_url, project_name)
prefixes += f"\nPREFIX ofn:<http://www.ontotext.com/sparql/functions/>"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1883    0  1883    0     0   9813      0 --:--:-- --:--:-- --:--:-- 10403


In [6]:
# Structurer les rues de Paris, et les zones selon notre modèle

# Vider le graphe `addr_graph_name_uri`
query0 = prefixes + f"""
DELETE {{ ?s ?p ?o }}
WHERE {{ GRAPH <{addr_graph_name_uri}> {{ ?s ?p ?o }} }}
"""

# Création des rues de Paris et ajout d'un lien owl:sameAs avec son équivalent sur Wikidata
query1 = prefixes + f"""
INSERT {{ GRAPH <{addr_graph_name_uri}> {{
  ?landmark a addr:Landmark;
           addr:isLandmarkType addr:Thoroughfare;
           addr:hasAttribute ?attrName;
           owl:sameAs ?street.
  ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute.
}} }}
WHERE {{
  {{ ?street p:P361 [ps:P361 wd:Q16024163]. }}
  UNION
  {{ ?street p:P361 [ps:P361 wd:Q107311481]. }}
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#LM_", STRUUID())), ?x) AS ?landmark)
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#AN_", STRUUID())), ?x) AS ?attrName)
  }}
"""

query2 = prefixes + f"""
INSERT {{ GRAPH <{addr_graph_name_uri}> {{
 ?landmark rdfs:label ?streetLabel;
           skos:altLabel ?streetAltLabel;
           addr:within ?loc ;
           addr:hasAttribute ?officialNameAttr.
}} }}
WHERE {{
  ?landmark a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; owl:sameAs ?street.
  ?street wdt:P131 ?loc.
  ?street rdfs:label ?streetLabel.
  FILTER (LANG(?streetLabel) = "fr")
  OPTIONAL {{?street skos:altLabel ?streetAltLabel. FILTER (LANG(?streetAltLabel) = "fr")}}
}}
"""

# Obtention des noms officiels des rues
query3 = prefixes + f"""
INSERT {{ GRAPH <{addr_graph_name_uri}> {{
  ?attrName addr:version ?attrNameVersion.
  ?attrNameVersion owl:sameAs ?officialNameSt; a addr:AttributeVersion; addr:value ?officialName.
  ?startEvent a addr:Event; addr:hasTime ?startTimeInstant.
  ?endEvent a addr:Event; addr:hasTime ?endTimeInstant.
  ?startChange a addr:AttributeChange; addr:isChangeType addr:NameChange; addr:dependsOn ?startEvent; addr:appliedTo ?attrName; addr:after ?attrNameVersion.
  ?endChange a addr:AttributeChange; addr:isChangeType addr:NameChange; addr:dependsOn ?endEvent; addr:appliedTo ?attrName; addr:before ?attrNameVersion.
  ?startTimeInstant a addr:TimeInstant; addr:timeStamp ?startDateValue; addr:timePrecision ?startDatePrec; addr:timeCalendar ?startDateCal.
  ?endTimeInstant a addr:TimeInstant; addr:timeStamp ?endDateValue; addr:timePrecision ?endDatePrec; addr:timeCalendar ?endDateCal.
}} }}
WHERE {{
  ?landmark a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; owl:sameAs [p:P1448 ?officialNameSt]; addr:hasAttribute ?attrName.
  ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute.
  ?officialNameSt ps:P1448 ?officialName.
  OPTIONAL{{?officialNameSt pqv:P580 [wikibase:timeValue ?startDateValue; wikibase:timePrecision ?startDatePrec; wikibase:timeCalendarModel ?startDateCal]}}
  OPTIONAL{{?officialNameSt pqv:P582 [wikibase:timeValue ?endDateValue; wikibase:timePrecision ?endDatePrec; wikibase:timeCalendarModel ?endDateCal]}}
  FILTER (LANG(?officialName) = "fr")
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#ANV_", STRUUID())), ?x) AS ?attrNameVersion)
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#EV_", STRUUID())), ?x) AS ?startEvent)
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#CH_", STRUUID())), ?x) AS ?startChange)
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#EV_", STRUUID())), ?x) AS ?endEvent)
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#CH_", STRUUID())), ?x) AS ?endChange)
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#TE_", STRUUID())), ?x) AS ?startTimeInstant)
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#TE_", STRUUID())), ?x) AS ?endTimeInstant)
}}
"""

# Obtention des dates de création et des dissolution des rues
query4 = prefixes + f"""
INSERT {{ GRAPH <{addr_graph_name_uri}> {{
  ?startEvent a addr:Event; addr:hasTime ?startTimeInstant.
  ?endEvent a addr:Event; addr:hasTime ?endTimeInstant.
  ?startChange a addr:LandmarkChange; addr:isChangeType addr:Creation; addr:appliedTo ?landmark; addr:dependsOn ?startEvent.
  ?endChange a addr:LandmarkChange; addr:isChangeType addr:Dissolution; addr:appliedTo ?landmark; addr:dependsOn ?endEvent.
  ?startTimeInstant a addr:TimeInstant; addr:timeStamp ?creationTimeValue; addr:timePrecision ?creationTimePrec; addr:timeCalendar ?creationTimeCal.
  ?endTimeInstant a addr:TimeInstant; addr:timeStamp ?dissolutionTimeValue; addr:timePrecision ?dissolutionTimePrec; addr:timeCalendar ?dissolutionTimeCal.
}} }}
WHERE {{
  ?landmark a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; owl:sameAs ?street.
  OPTIONAL {{ ?street p:P571 [psv:P571 [wikibase:timeValue ?creationTimeValue; wikibase:timePrecision ?creationTimePrec; wikibase:timeCalendarModel ?creationTimeCal]]. }}
  OPTIONAL {{ ?street p:P576 [psv:P576 [wikibase:timeValue ?dissolutionTimeValue; wikibase:timePrecision ?dissolutionTimePrec; wikibase:timeCalendarModel ?dissolutionTimeCal]]. }}
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#EV_", STRUUID())), ?x) AS ?startEvent)
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#CH_", STRUUID())), ?x) AS ?startChange)
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#EV_", STRUUID())), ?x) AS ?endEvent)
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#CH_", STRUUID())), ?x) AS ?endChange)
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#TE_", STRUUID())), ?x) AS ?startTimeInstant)
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#TE_", STRUUID())), ?x) AS ?endTimeInstant)
}}
"""

# Données relatives aux arrondissements de Paris
query5 = prefixes + f"""
INSERT {{ GRAPH <{addr_graph_name_uri}> {{
  ?landmark a addr:Landmark;
           addr:isLandmarkType addr:District;
           rdfs:label ?arrdtLabel;
           skos:altLabel ?arrdtAltLabel;
           addr:within wd:Q90;
           addr:startDate ?startDate;
           owl:sameAs ?arrdt.
}} }}
WHERE {{
  ?arrdt wdt:P31 wd:Q702842; p:P131 [ps:P131 wd:Q90]; rdfs:label ?arrdtLabel; skos:altLabel ?arrdtAltLabel.
  FILTER(LANG(?arrdtLabel) = "fr" && LANG(?arrdtAltLabel) = "fr")
  OPTIONAL {{?arrdt wdt:P571 ?startDate}}
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#LM_", STRUUID())), ?x) AS ?landmark)
}}
"""

# Données relatives aux quartiers administratifs de Paris
query6 = prefixes + f"""
INSERT {{ GRAPH <{addr_graph_name_uri}> {{
  ?landmark a addr:Landmark;
           addr:isLandmarkType addr:District;
           rdfs:label ?quartierLabel;
           skos:altLabel ?quartierAltLabel;
           addr:within ?loc; 
           owl:sameAs ?quartier.
}} }}
WHERE {{
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#LM_", STRUUID())), ?x) AS ?landmark)
  ?quartier wdt:P31 wd:Q252916; rdfs:label ?quartierLabel; skos:altLabel ?quartierAltLabel; p:P131 [ps:P131 ?loc].
  FILTER(LANG(?quartierLabel) = "fr" && LANG(?quartierAltLabel) = "fr")
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#LM_", STRUUID())), ?x) AS ?landmark)
}}
"""

# Données relatives à la ville de Paris
query7 = prefixes + f"""
INSERT {{ GRAPH <{addr_graph_name_uri}> {{
  ?landmark a addr:Landmark;
           addr:isLandmarkType addr:City;
           rdfs:label ?parisLabel;
           skos:altLabel ?parisAltLabel;
           owl:sameAs ?paris.
}} }}
WHERE {{
  BIND (wd:Q90 AS ?paris)
  ?paris rdfs:label ?parisLabel; skos:altLabel ?parisAltLabel.
  FILTER(LANG(?parisLabel) = "fr" && LANG(?parisAltLabel) = "fr")
  BIND(IF(1=1, URI(CONCAT("http://rdf.geohistoricaldata.org/address#LM_", STRUUID())), ?x) AS ?landmark)
}}
"""

wiki_queries = [query0, query1, query2, query3, query4, query5, query6, query7]

Création d'un graphe nommé stockant les rues selon l'ontologie définie à partir des données de Wikidata

In [7]:
for query in wiki_queries:
    gd.update_query(query, graphdb_url, project_name)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3164    0     0  100  3164      0   5611 --:--:-- --:--:-- --:--:--  5680
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3974    0     0  100  3974      0   1343  0:00:02  0:00:02 --:--:--  1345
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3805    0     0  100  3805      0   1511  0:00:02  0:00:02 --:--:--  1514
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6217    0     0  100  6217      0     92  0:01:07  0:01:07 --:--:--     0
  % Total    % Received % Xferd  Average Speed   Tim

#### Traitements après l'import de données

* `loc_query_1` : 
    * ajoute un élément de classe `addr:Attribute` relatif au nom de la voie qui permet d'avoir l'ensemble des noms officiels que cette dernière a porté durant son existence. Il est lié à la voie via le prédicat `addr:hasAttribute` ;
    * `BIND(URI(...) AS ?z)` est appelé deux fois afin que `?attrName` soit unique pour chaque rue (sinon il possède la même valeur tout le temps).
* `loc_query_2` : lors de la construction du graphe via les requêtes de la section précédente, les versions des noms sont liées directement aux voies (`?voie ?aPourVersionDeNom ?nomVersion`). Cette requête supprime ce triplet et associe `nomVersion` a l'élément crée grâce à `loc_query_1`. Ainsi, on a : `?voie ?aPourAttributDeNom ?nomAttribut. ?nomAttribut ?aPourVersionDeNom ?nomVersion.`
* `loc_query_3` : fusion d'éléments de type `addr:Event` qui sont équivalents.

In [9]:
loc_query_0 = prefixes + f"""
DELETE {{ 
    ?timeInstant addr:timePrecision ?timePrecision.
}}
INSERT {{ GRAPH <{addr_graph_name_uri}> {{
    ?timeInstant addr:timePrecision ?timeUnit.
}} }}
WHERE {{ GRAPH <{addr_graph_name_uri}> {{
    ?timeInstant a addr:TimeInstant; addr:timePrecision ?timePrecision.
        BIND(IF(?timePrecision = 11, time:unitDay, 
                IF(?timePrecision = 10, time:unitMonth,
                    IF(?timePrecision = 9, time:unitYear,
                        IF(?timePrecision = 8, time:unitDecade,
                            IF(?timePrecision = 7, time:unitCentury,
                                IF(?timePrecision = 6, time:unitMillenium, ?x
                            )))))) AS ?timeUnit)
    }} }}

"""

loc_query_1 = prefixes + f"""

DELETE {{
    ?change1 ?p ?o.
    ?event1 ?pe ?oe.
    ?se ?pe ?event1.
    ?timeInstant1 a addr:timeInstant; addr:timeStamp ?timeStamp1; addr:timePrecision ?timePrec1; addr:timeCalendar ?timeCal1.
}}
INSERT {{ GRAPH <{addr_graph_name_uri}> {{
    ?change2 addr:before ?attrNameVersion1.
}} }}
WHERE {{
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute; addr:version ?attrNameVersion1, ?attrNameVersion2.
    ?attrNameVersion1 a addr:AttributeVersion.
    ?attrNameVersion2 a addr:AttributeVersion.
    FILTER (?attrNameVersion1 != ?attrNameVersion2)
    ?change1 a addr:AttributeChange; addr:before ?attrNameVersion1; addr:dependsOn ?event1.
    ?event1 a addr:Event; addr:hasTime ?timeInstant1. 
    ?timeInstant1 a addr:TimeInstant; addr:timeStamp ?timeStamp1; addr:timePrecision ?timePrec1; addr:timeCalendar ?timeCal1.
    ?change2 a addr:AttributeChange; addr:after ?attrNameVersion2; addr:dependsOn [a addr:Event; addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeStamp2; addr:timePrecision ?timePrec2; addr:timeCalendar ?timeCal2]].
    BIND(ofn:asDays(?timeStamp2 - ?timeStamp1) AS ?diffTime)
    FILTER(?diffTime >= 0.0 && ?timeCal1 = ?timeCal2)
    FILTER ((?diffTime <= 1.0 && ?timePrec1 = time:unitDay && ?timePrec2 = time:unitDay) ||
         (?diffTime <= 0 && ?timePrec1 != time:unitDay && ?timePrec2 != time:unitDay))
    ?change1 ?p ?o.
    {{?event1 ?pe ?oe}}UNION{{?se ?pe ?event1}}
}}
"""

loc_query_2 = prefixes + f"""
DELETE {{
    ?creaEv ?p ?o.
    ?s ?p ?creaEv.
    ?crevEvTime a addr:TimeInstant; addr:timePrecision ?creaEvTimePrec; addr:timeStamp ?creaEvTimeVal; addr:timeCalendar ?creaEvTimeCal.
}}
INSERT {{ GRAPH <{addr_graph_name_uri}> {{
    ?lmCrea addr:dependsOn ?attrNameEv.
}} }}
WHERE {{
    ?lmCrea a addr:LandmarkChange; addr:isChangeType addr:Creation; addr:appliedTo ?landmark; addr:dependsOn ?creaEv.
    ?creaEv a addr:Event; addr:hasTime ?crevEvTime.
    ?crevEvTime a addr:TimeInstant; addr:timePrecision ?creaEvTimePrec; addr:timeStamp ?creaEvTimeVal; addr:timeCalendar ?creaEvTimeCal.
    ?landmark a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; addr:hasAttribute ?nameAttr.
    ?attrNameChange a addr:AttributeChange; addr:appliedTo ?nameAttr; addr:dependsOn ?attrNameEv.
    ?attrNameEv a addr:Event; addr:hasTime ?attrNameEvTime. 
    ?attrNameEvTime a addr:TimeInstant; addr:timePrecision ?newNameEvTimePrec; addr:timeStamp ?newNameEvTimeVal; addr:timeCalendar ?newNameEvTimeCal.
    FILTER(?attrNameEv != ?creaEv)
    BIND(ofn:asDays(?creaEvTimeVal - ?newNameEvTimeVal) AS ?diffTime)
    FILTER(?diffTime >= 0.0 && ?newNameEvTimeCal = ?creaEvTimeCal)
    FILTER ((?diffTime <= 1.0 && ?creaEvTimePrec = time:unitDay && ?newNameEvTimePrec = time:unitDay) ||
         (?diffTime <= 0 && ?creaEvTimePrec != time:unitDay && ?timePrec2 != time:newNameEvTimePrec))
        {{?creaEv ?p ?o}}UNION{{?s ?p ?creaEv}}
}}
"""

loc_queries = [loc_query_0, loc_query_1, loc_query_2]

Nettoyage des données en fusionnant certains événements et changements

In [10]:
for loc_query in loc_queries:
    gd.update_query(query, graphdb_url, project_name)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3853    0     0  100  3853      0   6356 --:--:-- --:--:-- --:--:--  6530
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3853    0     0  100  3853      0  14614 --:--:-- --:--:-- --:--:-- 14876
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3853    0     0  100  3853      0  14769 --:--:-- --:--:-- --:--:-- 15229


### Requêtes pour faire des sélections dans le graphe

In [ ]:
# Sélection des noms qu'a pris la place de la Nation durant sa vie
query = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

select ?street ?name ?timeValueStart ?timeValueEnd where { 
	?street a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; addr:hasAttribute ?attrName; (rdfs:label|skos:altLabel) "place de la Nation"@fr.
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute; addr:version ?attrNameVersion.
    ?attrNameVersion addr:value ?name.
    ?changeEnd addr:before ?attrNameVersion; addr:appliedTo ?attrName; addr:dependsOn ?eventEnd.
    ?changeStart addr:after ?attrNameVersion; addr:appliedTo ?attrName; addr:dependsOn ?eventStart.
    ?eventEnd a addr:Event.
    ?eventStart a addr:Event.
    OPTIONAL {?eventEnd addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValueEnd]}
    OPTIONAL {?eventEnd addr:hasTime [a addr:TimeInstant; addr:timePrecision ?timePrecisionEnd]}
    OPTIONAL {?eventEnd addr:hasTime [a addr:TimeInstant; addr:timeCalendar ?timeCalendarEnd]}
    OPTIONAL {?eventStart addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValueStart]}
    OPTIONAL {?eventStart addr:hasTime [a addr:TimeInstant; addr:timePrecision ?timePrecisionStart]}
    OPTIONAL {?eventStart addr:hasTime [a addr:TimeInstant; addr:timeCalendar ?timeCalendarStart]}
""" 

# Sélection de l'historique des noms de la place de la Nation avec les dates
query = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

select ?street ?change ?event ?timeValue ?timePrecision ?timeCalendar ?beforeVersion ?afterVersion where { 
	?street a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; addr:hasAttribute ?attrName; (rdfs:label|skos:altLabel) "place de la Nation"@fr.
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute.
    ?change addr:appliedTo ?attrName; addr:dependsOn ?event.
    ?event a addr:Event.
    OPTIONAL {?event addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValue]}
    OPTIONAL {?event addr:hasTime [a addr:TimeInstant; addr:timePrecision ?timePrecision]}
    OPTIONAL {?event addr:hasTime [a addr:TimeInstant; addr:timeCalendar ?timeCalendar]}
    OPTIONAL {?change addr:before [addr:value ?beforeVersion]}
    OPTIONAL {?change addr:after [addr:value ?afterVersion]}
}
"""

# Sélection du nom de la place de la Nation à une date donnée
query = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

select ?street ?name ?selectTime where { 
	?street a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; addr:hasAttribute ?attrName; (rdfs:label|skos:altLabel) "place de la Nation"@fr.
    BIND("1800-01-08T00:00:00"^^xsd:dateTime AS ?selectTime)
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute; addr:version ?attrNameVersion.
    ?attrNameVersion a addr:AttributeVersion; addr:value ?name.
    ?change1 addr:before ?attrNameVersion; addr:dependsOn ?event1.
    ?change2 addr:after ?attrNameVersion; addr:dependsOn ?event2.
    ?event1 a addr:Event.
    ?event2 a addr:Event.
    OPTIONAL {?event1 addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValue1]}
    OPTIONAL {?event1 addr:hasTime [a addr:TimeInstant; addr:timePrecision ?timePrecision1]}
    OPTIONAL {?event1 addr:hasTime [a addr:TimeInstant; addr:timeCalendar ?timeCalendar1]}
    OPTIONAL {?event2 addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValue2]}
    OPTIONAL {?event2 addr:hasTime [a addr:TimeInstant; addr:timePrecision ?timePrecision2]}
    OPTIONAL {?event2 addr:hasTime [a addr:TimeInstant; addr:timeCalendar ?timeCalendar2]}
    FILTER ((?timeValue1 >= ?selectTime && ?timeValue2 <= ?selectTime && ?timeCalendar1 = ?timeCalendar2) ||
        (?timeValue1 >= ?selectTime && !BOUND(?timeValue2)) ||
        (?timeValue2 <= ?selectTime && !BOUND(?timeValue1))
    ) 
}
""" 

# Sélection des voies existant à une date donnée avec leur nom d'époque
query = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

select ?street ?name where { 
    BIND("1900-01-08T00:00:00"^^xsd:dateTime AS ?selectTime)
	?street a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; addr:hasAttribute ?attrName.
    ?changeStreet1 addr:dependsOn ?eventStreet1; addr:isChangeType addr:Dissolution; addr:appliedTo ?street.
    ?changeStreet2 addr:dependsOn ?eventStreet2; addr:isChangeType addr:Creation; addr:appliedTo ?street.
    ?eventStreet1 a addr:Event.
    ?eventStreet2 a addr:Event.
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute; addr:version ?attrNameVersion.
    ?attrNameVersion a addr:AttributeVersion; addr:value ?name.
    ?change1 addr:before ?attrNameVersion; addr:dependsOn ?event1.
    ?change2 addr:after ?attrNameVersion; addr:dependsOn ?event2.
    ?event1 a addr:Event.
    ?event2 a addr:Event.
    OPTIONAL {?event1 addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValue1]}
    OPTIONAL {?event1 addr:hasTime [a addr:TimeInstant; addr:timePrecision ?timePrecision1]}
    OPTIONAL {?event1 addr:hasTime [a addr:TimeInstant; addr:timeCalendar ?timeCalendar1]}
    OPTIONAL {?event2 addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValue2]}
    OPTIONAL {?event2 addr:hasTime [a addr:TimeInstant; addr:timePrecision ?timePrecision2]}
    OPTIONAL {?event2 addr:hasTime [a addr:TimeInstant; addr:timeCalendar ?timeCalendar2]}
    OPTIONAL {?eventStreet1 addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValueStreet1]}
    OPTIONAL {?eventStreet1 addr:hasTime [a addr:TimeInstant; addr:timePrecision ?timePrecisionStreet1]}
    OPTIONAL {?eventStreet1 addr:hasTime [a addr:TimeInstant; addr:timeCalendar ?timeCalendarStreet1]}
    OPTIONAL {?eventStreet2 addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValueStreet2]}
    OPTIONAL {?eventStreet2 addr:hasTime [a addr:TimeInstant; addr:timePrecision ?timePrecisionStreet2]}
    OPTIONAL {?eventStreet2 addr:hasTime [a addr:TimeInstant; addr:timeCalendar ?timeCalendarStreet2]}
    FILTER ((?timeValue1 >= ?selectTime && ?timeValue2 <= ?selectTime && ?timeCalendar1 = ?timeCalendar2) ||
        (?timeValue1 >= ?selectTime && !BOUND(?timeValue2)) ||
        (?timeValue2 <= ?selectTime && !BOUND(?timeValue1))
    ) 
    FILTER ((?timeValueStreet1 >= ?selectTime && ?timeValueStreet2 <= ?selectTime && ?timeCalendarStreet1 = ?timeCalendarStreet2) ||
        (?timeValueStreet1 >= ?selectTime && !BOUND(?timeValueStreet2)) ||
        (?timeValueStreet2 <= ?selectTime && !BOUND(?timeValueStreet1))
    ) 
}
"""

## Ajout des emprises de voie

In [18]:
# ontorefine_cmd = "ontorefine-cli"
# ontorefine_cmd = "/opt/ontotext-refine/lib/app/bin/ontorefine-cli"
ontorefine_cmd = "/Applications/Ontotext\ Refine.app/Contents/app/bin/ontorefine-cli"
ontorefine_url = "http://localhost:7333"

sources = [
    {"filename": "andriveau_1849_am.csv", "mapping_file":"mapping_andriveau_1849_am.json","graphname":"andriveau1849", "date":"1849-01-01"},
    {"filename": "atlas_municipal_1888_am.csv", "mapping_file":"mapping_atlas_municipal_1888_am.json","graphname":"atlasMunicipal1888", "date":"1888-01-01"},
    {"filename": "piquet_1826_am.csv", "mapping_file":"mapping_piquet_1826_am.json","graphname":"piquet1826", "date":"1826-01-01"},
    {"filename": "verniquet_1791_am.csv", "mapping_file":"mapping_verniquet_1791_am.json","graphname":"verniquet1791", "date":"1791-01-01"},
]

query = """

"""

for source in sources:
    source_file = os.path.join(source_folder, source.get("filename"))
    mapping_file = os.path.join(mapping_folder, source.get("mapping_file"))
    export_file = os.path.join(tmp_folder, source.get("filename").replace(".csv", ".ttl"))
    graph_name = source.get("graphname")
    date = source.get("date")
    timestamp = f"+{date}T00:00:00"
    otr.get_export_file_from_ontorefine(source_file, mapping_file, export_file, ontorefine_cmd, ontorefine_url, project_name)
    gd.import_ttl_file_in_graphdb(graphdb_url, project_name, export_file, graph_name)

Successfully deleted project with identifier: 2687116559576


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   99k    0     0  100   99k      0   268k --:--:-- --:--:-- --:--:--  274k


Successfully deleted project with identifier: 2584342009058


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  109k    0     0  100  109k      0   272k --:--:-- --:--:-- --:--:--  278k


Successfully deleted project with identifier: 2431817778940


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 81112    0     0  100 81112      0   237k --:--:-- --:--:-- --:--:--  240k


Successfully deleted project with identifier: 1961714364148


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 71257    0     0  100 71257      0   226k --:--:-- --:--:-- --:--:--  230k
